# Skip-gram addr2vec

In this notebook, I'll convert addresses to vectors through TensorFlow and understand the ability to correct for mistyping.


## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [openaddress US North East dataset](https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip), and extract onto 'openaddr' directory if not found. read the csv files and load address dictionaries

In [2]:
import glob
import os
import pandas as pd

from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'openaddr'
dataset_filename = 'openaddr-collected-us_northeast.zip'
dataset_name = 'Openaddress Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)


id_to_address = {}
address_to_id = {}
i = 0
for state in os.listdir('./openaddr/us'):
    
    for filename in glob.glob('./openaddr/us/{}/*.csv'.format(state)):
        csv = pd.read_csv(filename)
        if i == 0:
            print("Column names available: {}".format(csv.columns))
            print("Example data: {}".format(csv.iloc[1]))

        stack = np.stack((csv['CITY'],), axis=-1)
        for j in stack:
            addr = " ".join([str(k).lower()
                             for k in j if not isinstance(k, type(np.nan))])
            if addr not in address_to_id and addr != '' and addr != ' ':
                id_to_address[i] = addr
                address_to_id[addr] = i
                i += 1

del csv

Column names available: Index(['LON', 'LAT', 'NUMBER', 'STREET', 'UNIT', 'CITY', 'DISTRICT', 'REGION',
       'POSTCODE', 'ID', 'HASH'],
      dtype='object')
Example data: LON                       -78.5994
LAT                        42.6937
NUMBER                        2393
STREET                    LEWIS RD
UNIT                           NaN
CITY                        Aurora
DISTRICT                       NaN
REGION                         NaN
POSTCODE                14139 9710
ID          1424892010000003022000
HASH              d6ba36e791e84371
Name: 1, dtype: object


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compi

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
print([i for i in address_to_id.keys()][:30])

['aurora', 'wales', 'orchard park', 'holland', 'boston', 'colden', 'concord', 'collins', 'north collins', 'sardinia', 'brant', 'evans', 'eden', 'hamburg', 'lackawanna', 'west seneca', 'elma', 'marilla', 'cheektowaga', 'lancaster', 'alden', 'grand island', 'amherst', 'clarence', 'newstead', 'tonawanda', 'buffalo', 'goshen', 'monroe', 'washingtonville']


In [4]:
print("Total addresses: {}".format(len(address_to_id)))

vocab_to_id = {}
int_to_vocab = {}
idx = 0
for address in address_to_id.keys():
    for j in range(len(address)):
        if address[j] not in vocab_to_id:
            vocab_to_id[address[j]] = idx
            int_to_vocab[idx] = address[j]
            idx += 1

vocab_size = len(vocab_to_id)
print("Total unique letters: {}".format(vocab_size))

Total addresses: 4155
Total unique letters: 45


In [5]:
features = []
labels = []

for address, idx in address_to_id.items():
    X = np.zeros(vocab_size)
    Y = np.zeros(len(address_to_id))
    for j in range(len(address)):
        X[vocab_to_id[address[j]]] += 1 
    
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    
    features.append(X)
    
    Y[idx] = 1
    labels.append(Y)
    
print("Example feature vector and label:")
print(features[500])
print(labels[500])

Example feature vector and label:
[1.000e-03 1.000e-03 5.010e-01 5.010e-01 1.000e-03 5.010e-01 1.001e+00
 1.000e-03 5.010e-01 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 5.010e-01 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 5.010e-01 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03]
[0. 0. 0. ... 0. 0. 0.]


In [6]:
# Size of the encoding layer (the hidden layer)
encoding_dim = 200
alpha = 0.1

# Input and target placeholders
inp_shape = vocab_size

inputs_ = tf.placeholder(tf.float32 ,(None, inp_shape), name='inputs')
targets_ = tf.placeholder(tf.float32 ,(None, len(address_to_id)), name='targets')

# Input layer is 45
x1 = tf.expand_dims(inputs_, axis=1)
x1 = tf.layers.conv1d(x1, filters=64, kernel_size=1, strides=1, padding='valid')
x1 = tf.nn.dropout(x1, 0.5)

relu1 = tf.maximum(alpha * x1, x1)
# 41x64

x2 = tf.layers.conv1d(relu1, filters=128, kernel_size=1, strides=1, padding='valid')
x2 = tf.nn.dropout(x2, 0.5)
relu2 = tf.maximum(alpha * x2, x2)
# 37x128

x3 = tf.layers.conv1d(relu2, filters=256, kernel_size=1, strides=1, padding='valid')
x3 = tf.nn.dropout(x3, 0.5)
relu3 = tf.maximum(alpha * x3, x3)
# 33x256

first_h_dim = tf.layers.dense(relu3, 200)
relu3 = tf.maximum(alpha * first_h_dim, first_h_dim)
encoded = tf.layers.dense(relu3, encoding_dim)

# Output layer logits
logits = tf.layers.dense(encoded, len(address_to_id), activation=None, name='outputs')
# Sigmoid output from logits
#decoded = tf.nn.sigmoid(logits, name='outputs')

# loss = tf.log(tf.reduce_sum(tf.abs(targets_ - logits)) + 1)
loss = tf.nn.softmax_cross_entropy_with_logits(labels=targets_, logits=logits)
# Mean of the loss
cost = tf.reduce_mean(loss)




In [7]:
# Adam optimizer
opt = tf.train.AdamOptimizer(0.0005).minimize(cost)

# Create the session
sess = tf.Session()

epochs = 2000
batch_size = 128
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range((len(features)//batch_size) - 1):
        batch = features[ii*batch_size:(ii + 1)*batch_size]
        
        # add random noise
        for jj in batch:
            jj[np.random.randint(vocab_size)] += 0.05
        
        targ = np.array(labels[ii*batch_size:(ii + 1)*batch_size])

        feed = {inputs_: batch, targets_: targ}
        batch_cost, _ = sess.run([cost, opt], feed_dict=feed)

        print("Epoch: {}/{}...".format(e+1, epochs),
              "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/2000... Training loss: 8.3289
Epoch: 1/2000... Training loss: 8.3322
Epoch: 1/2000... Training loss: 8.3343
Epoch: 1/2000... Training loss: 8.3421
Epoch: 1/2000... Training loss: 8.3346
Epoch: 1/2000... Training loss: 8.3396
Epoch: 1/2000... Training loss: 8.3348
Epoch: 1/2000... Training loss: 8.3435
Epoch: 1/2000... Training loss: 8.3437
Epoch: 1/2000... Training loss: 8.3407
Epoch: 1/2000... Training loss: 8.3367
Epoch: 1/2000... Training loss: 8.3413
Epoch: 1/2000... Training loss: 8.3525
Epoch: 1/2000... Training loss: 8.3484
Epoch: 1/2000... Training loss: 8.3520
Epoch: 1/2000... Training loss: 8.3542
Epoch: 1/2000... Training loss: 8.3533
Epoch: 1/2000... Training loss: 8.3561
Epoch: 1/2000... Training loss: 8.3507
Epoch: 1/2000... Training loss: 8.3539
Epoch: 1/2000... Training loss: 8.3552
Epoch: 1/2000... Training loss: 8.3506
Epoch: 1/2000... Training loss: 8.3508
Epoch: 1/2000... Training loss: 8.3510
Epoch: 1/2000... Training loss: 8.3469
Epoch: 1/2000... Training

Epoch: 7/2000... Training loss: 8.2658
Epoch: 7/2000... Training loss: 8.2407
Epoch: 7/2000... Training loss: 8.2031
Epoch: 7/2000... Training loss: 8.2225
Epoch: 7/2000... Training loss: 8.2150
Epoch: 8/2000... Training loss: 8.2150
Epoch: 8/2000... Training loss: 8.2197
Epoch: 8/2000... Training loss: 8.2380
Epoch: 8/2000... Training loss: 8.2271
Epoch: 8/2000... Training loss: 8.2029
Epoch: 8/2000... Training loss: 8.2145
Epoch: 8/2000... Training loss: 8.2029
Epoch: 8/2000... Training loss: 8.2105
Epoch: 8/2000... Training loss: 8.2018
Epoch: 8/2000... Training loss: 8.1402
Epoch: 8/2000... Training loss: 8.0856
Epoch: 8/2000... Training loss: 8.1832
Epoch: 8/2000... Training loss: 8.2264
Epoch: 8/2000... Training loss: 8.2322
Epoch: 8/2000... Training loss: 8.2161
Epoch: 8/2000... Training loss: 8.2163
Epoch: 8/2000... Training loss: 8.2052
Epoch: 8/2000... Training loss: 8.1549
Epoch: 8/2000... Training loss: 8.0876
Epoch: 8/2000... Training loss: 8.0671
Epoch: 8/2000... Training

Epoch: 14/2000... Training loss: 6.9821
Epoch: 14/2000... Training loss: 7.2592
Epoch: 14/2000... Training loss: 7.1720
Epoch: 14/2000... Training loss: 7.0675
Epoch: 14/2000... Training loss: 7.3492
Epoch: 14/2000... Training loss: 7.0549
Epoch: 14/2000... Training loss: 6.9941
Epoch: 14/2000... Training loss: 6.7749
Epoch: 14/2000... Training loss: 6.9314
Epoch: 14/2000... Training loss: 7.0359
Epoch: 14/2000... Training loss: 7.0308
Epoch: 14/2000... Training loss: 6.9854
Epoch: 14/2000... Training loss: 6.9090
Epoch: 14/2000... Training loss: 6.8549
Epoch: 15/2000... Training loss: 6.8899
Epoch: 15/2000... Training loss: 6.6352
Epoch: 15/2000... Training loss: 6.7345
Epoch: 15/2000... Training loss: 6.7667
Epoch: 15/2000... Training loss: 6.7725
Epoch: 15/2000... Training loss: 6.8042
Epoch: 15/2000... Training loss: 6.7322
Epoch: 15/2000... Training loss: 6.9045
Epoch: 15/2000... Training loss: 6.6731
Epoch: 15/2000... Training loss: 6.8823
Epoch: 15/2000... Training loss: 7.1366


Epoch: 21/2000... Training loss: 5.9568
Epoch: 21/2000... Training loss: 6.3130
Epoch: 21/2000... Training loss: 6.2530
Epoch: 21/2000... Training loss: 6.1378
Epoch: 21/2000... Training loss: 5.9577
Epoch: 21/2000... Training loss: 6.0485
Epoch: 21/2000... Training loss: 5.9834
Epoch: 21/2000... Training loss: 5.9615
Epoch: 21/2000... Training loss: 5.9164
Epoch: 21/2000... Training loss: 5.8780
Epoch: 21/2000... Training loss: 6.0509
Epoch: 21/2000... Training loss: 5.8714
Epoch: 21/2000... Training loss: 5.7197
Epoch: 21/2000... Training loss: 6.0167
Epoch: 21/2000... Training loss: 6.0350
Epoch: 21/2000... Training loss: 5.8057
Epoch: 21/2000... Training loss: 5.8424
Epoch: 21/2000... Training loss: 6.0925
Epoch: 21/2000... Training loss: 5.9040
Epoch: 21/2000... Training loss: 6.1310
Epoch: 21/2000... Training loss: 5.9517
Epoch: 21/2000... Training loss: 5.9562
Epoch: 21/2000... Training loss: 5.9311
Epoch: 22/2000... Training loss: 6.0913
Epoch: 22/2000... Training loss: 5.9322


Epoch: 27/2000... Training loss: 5.5542
Epoch: 28/2000... Training loss: 5.2918
Epoch: 28/2000... Training loss: 5.2977
Epoch: 28/2000... Training loss: 5.3845
Epoch: 28/2000... Training loss: 5.3129
Epoch: 28/2000... Training loss: 5.3487
Epoch: 28/2000... Training loss: 5.2764
Epoch: 28/2000... Training loss: 5.3687
Epoch: 28/2000... Training loss: 5.4652
Epoch: 28/2000... Training loss: 5.5003
Epoch: 28/2000... Training loss: 5.6866
Epoch: 28/2000... Training loss: 5.6434
Epoch: 28/2000... Training loss: 5.6801
Epoch: 28/2000... Training loss: 5.4957
Epoch: 28/2000... Training loss: 5.3885
Epoch: 28/2000... Training loss: 5.7060
Epoch: 28/2000... Training loss: 5.5060
Epoch: 28/2000... Training loss: 5.3955
Epoch: 28/2000... Training loss: 5.4282
Epoch: 28/2000... Training loss: 5.5651
Epoch: 28/2000... Training loss: 5.4739
Epoch: 28/2000... Training loss: 5.3082
Epoch: 28/2000... Training loss: 5.4160
Epoch: 28/2000... Training loss: 5.7078
Epoch: 28/2000... Training loss: 5.5102


...

Epoch: 1973/2000... Training loss: 0.3379
Epoch: 1973/2000... Training loss: 0.3866
Epoch: 1973/2000... Training loss: 0.2506
Epoch: 1973/2000... Training loss: 0.4475
Epoch: 1973/2000... Training loss: 0.4011
Epoch: 1973/2000... Training loss: 0.3961
Epoch: 1973/2000... Training loss: 0.4198
Epoch: 1973/2000... Training loss: 0.5194
Epoch: 1973/2000... Training loss: 0.4261
Epoch: 1973/2000... Training loss: 0.3288
Epoch: 1973/2000... Training loss: 0.3954
Epoch: 1973/2000... Training loss: 0.4012
Epoch: 1974/2000... Training loss: 0.3663
Epoch: 1974/2000... Training loss: 0.3980
Epoch: 1974/2000... Training loss: 0.3409
Epoch: 1974/2000... Training loss: 0.4650
Epoch: 1974/2000... Training loss: 0.3662
Epoch: 1974/2000... Training loss: 0.4222
Epoch: 1974/2000... Training loss: 0.2341
Epoch: 1974/2000... Training loss: 0.2485
Epoch: 1974/2000... Training loss: 0.3942
Epoch: 1974/2000... Training loss: 0.3551
Epoch: 1974/2000... Training loss: 0.4236
Epoch: 1974/2000... Training loss:

Epoch: 1979/2000... Training loss: 0.3007
Epoch: 1979/2000... Training loss: 0.4186
Epoch: 1980/2000... Training loss: 0.3316
Epoch: 1980/2000... Training loss: 0.3134
Epoch: 1980/2000... Training loss: 0.4620
Epoch: 1980/2000... Training loss: 0.4444
Epoch: 1980/2000... Training loss: 0.2902
Epoch: 1980/2000... Training loss: 0.4988
Epoch: 1980/2000... Training loss: 0.2835
Epoch: 1980/2000... Training loss: 0.3454
Epoch: 1980/2000... Training loss: 0.3481
Epoch: 1980/2000... Training loss: 0.3212
Epoch: 1980/2000... Training loss: 0.5694
Epoch: 1980/2000... Training loss: 0.3916
Epoch: 1980/2000... Training loss: 0.5440
Epoch: 1980/2000... Training loss: 0.2560
Epoch: 1980/2000... Training loss: 0.4809
Epoch: 1980/2000... Training loss: 0.5998
Epoch: 1980/2000... Training loss: 0.1983
Epoch: 1980/2000... Training loss: 0.5246
Epoch: 1980/2000... Training loss: 0.3395
Epoch: 1980/2000... Training loss: 0.2680
Epoch: 1980/2000... Training loss: 0.2792
Epoch: 1980/2000... Training loss:

Epoch: 1986/2000... Training loss: 0.3599
Epoch: 1986/2000... Training loss: 0.2932
Epoch: 1986/2000... Training loss: 0.3627
Epoch: 1986/2000... Training loss: 0.4642
Epoch: 1986/2000... Training loss: 0.3378
Epoch: 1986/2000... Training loss: 0.3800
Epoch: 1986/2000... Training loss: 0.3104
Epoch: 1986/2000... Training loss: 0.2802
Epoch: 1986/2000... Training loss: 0.4288
Epoch: 1986/2000... Training loss: 0.5661
Epoch: 1986/2000... Training loss: 0.3161
Epoch: 1986/2000... Training loss: 0.1908
Epoch: 1986/2000... Training loss: 0.3786
Epoch: 1986/2000... Training loss: 0.3208
Epoch: 1986/2000... Training loss: 0.3098
Epoch: 1986/2000... Training loss: 0.3885
Epoch: 1986/2000... Training loss: 0.5970
Epoch: 1986/2000... Training loss: 0.3260
Epoch: 1986/2000... Training loss: 0.4788
Epoch: 1986/2000... Training loss: 0.3786
Epoch: 1986/2000... Training loss: 0.5419
Epoch: 1986/2000... Training loss: 0.3762
Epoch: 1986/2000... Training loss: 0.2660
Epoch: 1987/2000... Training loss:

Epoch: 1992/2000... Training loss: 0.3459
Epoch: 1992/2000... Training loss: 0.3202
Epoch: 1992/2000... Training loss: 0.3431
Epoch: 1992/2000... Training loss: 0.2807
Epoch: 1992/2000... Training loss: 0.2528
Epoch: 1992/2000... Training loss: 0.3776
Epoch: 1992/2000... Training loss: 0.4161
Epoch: 1992/2000... Training loss: 0.3798
Epoch: 1992/2000... Training loss: 0.3654
Epoch: 1992/2000... Training loss: 0.2563
Epoch: 1992/2000... Training loss: 0.4219
Epoch: 1992/2000... Training loss: 0.4583
Epoch: 1992/2000... Training loss: 0.3306
Epoch: 1993/2000... Training loss: 0.2002
Epoch: 1993/2000... Training loss: 0.2764
Epoch: 1993/2000... Training loss: 0.3296
Epoch: 1993/2000... Training loss: 0.3601
Epoch: 1993/2000... Training loss: 0.3972
Epoch: 1993/2000... Training loss: 0.3904
Epoch: 1993/2000... Training loss: 0.5179
Epoch: 1993/2000... Training loss: 0.3138
Epoch: 1993/2000... Training loss: 0.3304
Epoch: 1993/2000... Training loss: 0.5233
Epoch: 1993/2000... Training loss:

Epoch: 1998/2000... Training loss: 0.3138
Epoch: 1998/2000... Training loss: 0.2799
Epoch: 1998/2000... Training loss: 0.2966
Epoch: 1999/2000... Training loss: 0.3532
Epoch: 1999/2000... Training loss: 0.4194
Epoch: 1999/2000... Training loss: 0.2748
Epoch: 1999/2000... Training loss: 0.4170
Epoch: 1999/2000... Training loss: 0.3003
Epoch: 1999/2000... Training loss: 0.4280
Epoch: 1999/2000... Training loss: 0.4209
Epoch: 1999/2000... Training loss: 0.3241
Epoch: 1999/2000... Training loss: 0.2076
Epoch: 1999/2000... Training loss: 0.4180
Epoch: 1999/2000... Training loss: 0.2817
Epoch: 1999/2000... Training loss: 0.2936
Epoch: 1999/2000... Training loss: 0.2890
Epoch: 1999/2000... Training loss: 0.3955
Epoch: 1999/2000... Training loss: 0.3456
Epoch: 1999/2000... Training loss: 0.6288
Epoch: 1999/2000... Training loss: 0.2777
Epoch: 1999/2000... Training loss: 0.3146
Epoch: 1999/2000... Training loss: 0.3833
Epoch: 1999/2000... Training loss: 0.3945
Epoch: 1999/2000... Training loss:

In [8]:
test_words = ['aulola', 'wale', 'orchard park', 'holland', 'boston', 'colden', 
              'concord', 'collins', 'north collins', 'sardinia', 'brant', 'evans', 'eden',
              'hamburg', 'lackawanna', 'west seneca', 'elma', 'marilla', 'cheektowaga', 
              'lancapter', 'aldest', 'grant islands', 'amherst', 'clarence', 'newstead', 
              'tonawanda', 'bufalo', 'goshen ny', 'monroe ny']

in_words = []
for word in test_words:
    X = np.zeros(vocab_size)
    for j in range(len(word)):
        if word[j] in vocab_to_id:
            X[vocab_to_id[word[j]]] += 1
            
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    in_words.append(X)
    
in_words = np.array(in_words)

reconstructed, compressed = sess.run([logits, encoded], feed_dict={inputs_: in_words})

out_words = []
for out, inp in zip(reconstructed, test_words):
    best_match = np.argmax(out)
    
    out_words.append((id_to_address[best_match], inp))
    
        
print(out_words)

[('lawrence', 'aulola'), ('walker valley', 'wale'), ('valencia boro', 'orchard park'), ('rivervale', 'holland'), ('toboyne', 'boston'), ('enfield', 'colden'), ('laporte boro', 'concord'), ('huntingdon valley', 'collins'), ('new berlin', 'north collins'), ('upper tulpehocken', 'sardinia'), ('bayonne', 'brant'), ('great barrington', 'evans'), ('alden', 'eden'), ('upperstrasburg', 'hamburg'), ('coaldale boro', 'lackawanna'), ('beaver falls', 'west seneca'), ('sunapee', 'elma'), ('hamlin', 'marilla'), ('beaver falls', 'cheektowaga'), ('glen gardner', 'lancapter'), ('friendship', 'aldest'), ('osceola', 'grant islands'), ('rosendale', 'amherst'), ('hoosick', 'clarence'), ('beaver falls', 'newstead'), ('andes', 'tonawanda'), ('woodcock twp', 'bufalo'), ('landenburg', 'goshen ny'), ('london', 'monroe ny')]
